In [39]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path
from keras.preprocessing.image import img_to_array

from keras.datasets import mnist
import numpy as np
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path

from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.utils import to_categorical

import tensorflow as tf

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from tensorflow.keras.regularizers import l2

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout

from scipy.ndimage import gaussian_filter


# Data Cleaning & Processing

In [217]:
# importing and resizing image data 
# change path to wherever your data lives on your computer
csv_path = "/Users/madelinewhitlow/Desktop/MIDS_W207/petfinder-pawpularity-score/train.csv"
image_path = '/Users/madelinewhitlow/Desktop/MIDS_W207/petfinder-pawpularity-score/train/'

def CutenessBuckets(pawpularity_score):
    if pawpularity_score <= 10:
        return 0
    elif pawpularity_score <= 20:
        return 1
    elif pawpularity_score <= 30:
        return 2
    elif pawpularity_score <= 40:
        return 3
    elif pawpularity_score <= 50:
        return 4
    elif pawpularity_score <= 60:
        return 5
    elif pawpularity_score <= 70:
        return 6
    elif pawpularity_score <= 80:
        return 7
    elif pawpularity_score <= 90:
        return 8
    elif pawpularity_score <= 100:
        return 9

def CleanandProcessData(csv_path, image_path, image_resize):

    # imnporting csv with image ids and pawpularity scores
    df = pd.read_csv(csv_path)

    # creating path to image using the id in the data frame we've created
    df['path'] = image_path + df['Id'] + '.jpg'
    df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((image_resize,image_resize)))
    
    df['cuteness_bin'] = df['Pawpularity'].apply(CutenessBuckets)
    df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

    labels_raw=[]
    labels_bin = []
    imgArr=[]

    for ind in df.index:
        img = df.img_array[0]
        imgArr.append(img)
        labels_raw.append(df['Pawpularity'][ind])
        labels_bin.append(df['cuteness_bin'][ind])

    labels_raw = np.array(labels_raw)
    labels_bin = np.array(labels_bin)
    X_train=np.array(imgArr, dtype=float)
    print('Training Shape:',X_train.shape)
    print('Labels Shape;', labels.shape)
    return X_train, labels_raw, labels_bin

X_train, labels_raw, labels_bin = CleanandProcessData(csv_path, image_path, 20)

Training Shape: (9912, 20, 20, 3)
Labels Shape; (9912,)


In [218]:
# Shuffling data and separating into train, test and dev sets
shuffle = np.random.permutation(np.arange(X_train.shape[0]))

X_train, labels_raw, labels_bin = X_train[shuffle], labels_raw[shuffle], labels_bin[shuffle]

print('data shape: ', X_train.shape)
print('pawpularity label shape:', labels_raw.shape)
print('bin label shape:', labels_bin.shape)

# Set some variables to hold test, dev, and training data.
train_data, train_labels_raw, train_labels_bin = X_train[:7921], labels_raw[:7921], labels_bin[:7921]
test_data, test_labels_raw, test_labels_bin = X_train[8916:], labels_raw[8916:], labels_bin[8916:]
dev_data, dev_labels_raw, dev_labels_bin = X_train[7920:8916], labels_raw[7920:8916], labels_bin[7920:8916]

# raw labels (ie original pawpularity score)
test_labels_category = to_categorical(test_labels_raw)
train_labels_category = to_categorical(train_labels_raw)
dev_labels_category = to_categorical(dev_labels_raw)

# bucketed labels 
test_labels_bins_category = to_categorical(test_labels_bin)
train_labels_bins_category = to_categorical(train_labels_bin)
dev_labels_bins_category = to_categorical(dev_labels_bin)

# blurred image data (optional use)
blurred_train = gaussian_filter(train_data, sigma=1)
blurred_test = gaussian_filter(test_data, sigma = 1)

data shape:  (9912, 20, 20, 3)
pawpularity label shape: (9912,)
bin label shape: (9912,)


# First CCN Model 

In [219]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 101
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
model.fit(train_data,train_labels_category, validation_data=(test_data, test_labels_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
Epoch 1/3
248/248 [==============================] - 4s 13ms/step - loss: 4.2646 - accuracy: 0.0276 - val_loss: 4.1839 - val_accuracy: 0.0371
Epoch 2/3
248/248 [==============================] - 3s 13ms/step - loss: 4.2201 - accuracy: 0.0274 - val_loss: 4.1711 - val_accuracy: 0.0371
Epoch 3/3
32/32 [==============================] - 0s 3ms/step - loss: 4.1668 - accuracy: 0.0371

Test loss: 4.166837692260742
Test accuracy: 0.03714859485626221


# Same Model but using 10 category buckets for scores

In [221]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 10
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
model.fit(train_data,train_labels_bins_category, validation_data=(test_data, test_labels_bins_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
Epoch 1/3
248/248 [==============================] - 4s 15ms/step - loss: 1.9893 - accuracy: 0.2630 - val_loss: 1.9566 - val_accuracy: 0.2741
Epoch 2/3
248/248 [==============================] - 3s 11ms/step - loss: 1.9794 - accuracy: 0.2718 - val_loss: 1.9935 - val_accuracy: 0.2741
Epoch 3/3
32/32 [==============================] - 0s 3ms/step - loss: 1.9479 - accuracy: 0.2741

Test loss: 1.9479047060012817
Test accuracy: 0.27409639954566956


# Second CNN Model - Using raw paw scores

In [222]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")
history = model.fit(
    data_augmentation.flow(train_data,train_labels_category,batch_size=32),
    validation_data = (test_data, test_labels_category),
    steps_per_epoch = len(blurred) // 32,
    epochs = 3
)
score = model.evaluate(test_data, test_labels_category, verbose=1)
print('Test accuracy:', score[1])

Epoch 1/3
247/247 [==============================] - 9s 31ms/step - loss: 0.4597 - rmse: 0.5130 - mae: 0.3820 - mape: 372008864.0000 - val_loss: 0.2012 - val_rmse: 0.1081 - val_mae: 0.0621 - val_mape: 52739960.0000
Epoch 2/3
247/247 [==============================] - 7s 27ms/step - loss: 0.2352 - rmse: 0.2199 - mae: 0.1581 - mape: 148278960.0000 - val_loss: 0.1958 - val_rmse: 0.1086 - val_mae: 0.0633 - val_mape: 53961764.0000
Epoch 3/3
32/32 [==============================] - 0s 4ms/step - loss: 0.1875 - rmse: 0.0991 - mae: 0.0159 - mape: 6022475.5000
Test accuracy: 0.09908347576856613


# Second CNN Model - Using bucket paw scores

In [223]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")
history = model.fit(
    data_augmentation.flow(train_data,train_labels_bins_category,batch_size=32),
    validation_data = (test_data, test_labels_bins_category),
    steps_per_epoch = len(blurred) // 32,
    epochs = 3
)
score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
print('Test accuracy:', score[1])

Epoch 1/3
247/247 [==============================] - 8s 28ms/step - loss: 0.5331 - rmse: 0.5793 - mae: 0.4355 - mape: 343367328.0000 - val_loss: 0.2822 - val_rmse: 0.3019 - val_mae: 0.1528 - val_mape: 59382148.00005248
Epoch 2/3
247/247 [==============================] - 6s 26ms/step - loss: 0.3140 - rmse: 0.3545 - mae: 0.2403 - mape: 149989936.0000 - val_loss: 0.2759 - val_rmse: 0.3007 - val_mae: 0.1633 - val_mape: 71237664.0000
Epoch 3/3
32/32 [==============================] - 0s 5ms/step - loss: 0.2687 - rmse: 0.3001 - mae: 0.1744 - mape: 83679408.0000
Test accuracy: 0.3000822365283966


# Trying other models (Naive Bayes & KNN) - raw paw scores

In [180]:
# Other model attempts:
# have to reshape the data to input it into a different model
train_size = train_data.shape[0]
train_shape = train_data.shape[1]*train_data.shape[2]*train_data.shape[3]

test_size = test_data.shape[0]
test_shape = test_data.shape[1]*test_data.shape[2]*test_data.shape[3]

train_data_temp = train_data.reshape(train_size,train_shape)
test_data_temp = test_data.reshape(test_size,test_shape)
print('-'*10, ' K Nearest Neighbor ', '-'*10)
for k in [1,2,3,5,7]:
    knnmodel = KNeighborsClassifier(n_neighbors=k)
    knnmodel.fit(train_data_temp, train_labels_raw)
    test_predicted_labels = knnmodel.predict(test_data_temp)
    print("accuracy with k=", k, " : ", metrics.accuracy_score(test_labels_raw, test_predicted_labels))


print('-'*10, 'Naive Bayes ', '-'*10)
naive_bayes_model1 = BernoulliNB()
naive_bayes_model1.fit(train_data_temp, train_labels_raw)
print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels_raw))

----------  K Nearest Neighbor  ----------
accuracy with k= 1  :  0.0050200803212851405
accuracy with k= 2  :  0.015060240963855422
accuracy with k= 3  :  0.0321285140562249
accuracy with k= 5  :  0.01606425702811245
accuracy with k= 7  :  0.01606425702811245
---------- Naive Bayes  ----------
Bernoulli Model accuracy: 0.0231


In [228]:

print('-'*10, ' K Nearest Neighbor ', '-'*10)
for k in [1,2,3,5,7]:
    knnmodel = KNeighborsClassifier(n_neighbors=k)
    knnmodel.fit(train_data_temp, train_labels_bin)
    test_predicted_labels = knnmodel.predict(test_data_temp)
    print("accuracy with k=", k, " : ", metrics.accuracy_score(test_labels_bin, test_predicted_labels))


print('-'*10, 'Naive Bayes ', '-'*10)
naive_bayes_model1 = BernoulliNB()
naive_bayes_model1.fit(train_data_temp, train_labels_bin)
print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels_bin))

----------  K Nearest Neighbor  ----------
accuracy with k= 1  :  0.2740963855421687
accuracy with k= 2  :  0.2740963855421687
accuracy with k= 3  :  0.2520080321285141
accuracy with k= 5  :  0.2740963855421687
accuracy with k= 7  :  0.2740963855421687
---------- Naive Bayes  ----------
Bernoulli Model accuracy: 0.2741
